In [2]:
import xml.etree.ElementTree as ET

from py_constants import NAMESPACE_CPP, NAMESPACE_SRC

## Get the name of the packages used

In [86]:
def get_package_names_used_in_code(root):
    # Find all cpp:file elements and extract their text content
    cpp_files = [file_element.text for file_element in root.findall('.//cpp:file', namespaces=NAMESPACE_CPP)]
    return cpp_files

# Example usage
#cpp_files = get_package_names_used_in_code(root)
#for file_content in cpp_files:
#    print(file_content)

<iostream>
<cmath>


## Get the name of the Functions

In [87]:
def extract_function_names_from_code(root):
    features = {'names': []}
    name_elems = root.findall('.//src:function/src:name', namespaces=NAMESPACE_SRC)
    for name_elem in name_elems:
        features['names'].append(name_elem.text)

    return features

# Example usage
#extracted_features = extract_function_names_from_code(root)
#print("Names:", extracted_features['names'])

Names: ['isPrime', 'factorial', 'fibonacci', 'main']


## 1. Number of Functions (NF)

In [88]:
def count_function_declarations(root):
    # Count the number of function names within function elements
    count = sum(1 for _ in root.findall('.//src:function/src:name', namespaces=NAMESPACE_SRC))
    return count

# Example usage
#function_declaration_count = count_function_declarations(root)
#print(f"Total number of function declarations: {function_declaration_count}")

Total number of function declarations: 4


## Number of Functions (NF) excluding main()

In [90]:
def count_function_declarations_excluding_main(root):
    count = 0
    # Find all function names within function elements
    for function_name in root.findall('.//src:function/src:name', namespaces=NAMESPACE_SRC):
        # Check if the function name is not equal to "main"
        if function_name.text != "main":
            count += 1
    return count

# Example usage
#function_declaration_count = count_function_declarations_excluding_main(root)
#print(f"Total number of function declarations (excluding main): {function_declaration_count}")

Total number of function declarations (excluding main): 3


## 3. Number of Parameters (NP) in a function

In [91]:
def get_parameter_count_in_functions(root):
    function_parameter_counts = {}
    # Iterate through all function elements
    for function in root.findall('.//src:function', namespaces=NAMESPACE_SRC):
        function_name = function.find("./src:name", namespaces=NAMESPACE_SRC).text
        parameter_list = function.find("./src:parameter_list", namespaces=NAMESPACE_SRC)
        # Count the number of parameters in the parameter list
        parameter_count = sum(1 for _ in parameter_list.findall('./src:parameter', namespaces=NAMESPACE_SRC))
        # Store the parameter count for the function
        function_parameter_counts[function_name] = parameter_count
    #return function_parameter_counts
    return sum(function_parameter_counts.values())

# Example usage
#parameter_counts = get_parameter_count_in_functions(root)
#for function_name, parameter_count in parameter_counts.items():
#    print(f"Function {function_name} has {parameter_count} parameters.")
#total_count = sum(parameter_counts.values())
#print("Total count:", total_count)

Function isPrime has 1 parameters.
Function factorial has 1 parameters.
Function fibonacci has 1 parameters.
Function main has 0 parameters.
Total count: 3


## 7. Number of Simple Pointers (NPt-s)

In [33]:
def count_simple_pointers(root):
    simple_pointer_count = 0
    # Find all parameter elements
    parameters = root.findall('.//src:parameter', namespaces=NAMESPACE_SRC)
    # Iterate over each parameter
    for parameter in parameters:
        # Check if the parameter has a modifier tag with text '*'
        modifier = parameter.find('.//src:modifier', namespaces=NAMESPACE_SRC)
        if modifier is not None and modifier.text == '*':
            simple_pointer_count += 1
    return simple_pointer_count

# Example usage
#simple_pointer_count = count_simple_pointers(root)
#print(f"Number of simple pointers (NPt-s): {simple_pointer_count}")

Number of simple pointers (NPt-s): 5


## Number of Simple Pointers (NPt-s) by Function name

In [95]:
def count_simple_pointers_by_fun_name(root):
    # Initialize dictionary to store count of pointers for each function
    function_pointers = {}

    # Find all function elements
    functions = root.findall('.//src:function', namespaces=NAMESPACE_SRC)
    # Iterate over each function
    for function in functions:
        # Extract function name
        function_name = function.find('./src:name', namespaces=NAMESPACE_SRC).text 
        # Initialize count for simple pointers for this function
        simple_pointer_count = 0
        # Find all parameter elements within this function
        parameters = function.findall('.//src:parameter', namespaces=NAMESPACE_SRC)
        # Iterate over each parameter
        for parameter in parameters:
            # Check if the parameter has a modifier tag with text '*'
            modifier = parameter.find('.//src:modifier', namespaces=NAMESPACE_SRC)
            if modifier is not None and modifier.text == '*':
                simple_pointer_count += 1
        # Store the count of simple pointers for this function
        function_pointers[function_name] = simple_pointer_count
    return function_pointers

# Example usage
#function_pointers = count_simple_pointers_by_fun_name(root)
#for function_name, pointer_count in function_pointers.items():
#    print(f"{function_name} has {pointer_count} pointers")

func1 has 2 pointers
func2 has 2 pointers
func3 has 1 pointers
main has 0 pointers


## 8. Number of Simple Address (NAdd-s)

In [37]:
def count_simple_addresses(root):
    # Initialize count for simple addresses
    simple_address_count = 0
    # Find all function calls
    function_calls = root.findall('.//src:call', namespaces=NAMESPACE_SRC)
    # Iterate over each function call
    for call in function_calls:
        # Extract arguments from the call
        arguments = call.findall('.//src:argument', namespaces=NAMESPACE_SRC)
        # Iterate over each argument
        for argument in arguments:
            # Check if the argument has an address operator (&)
            address_operator = argument.find('.//src:operator', namespaces=NAMESPACE_SRC)
            #print("printing: ",address_operator)
            if address_operator is not None and address_operator.text == '&' :
                simple_address_count += 1
    return simple_address_count

# Example usage
#simple_address_count = count_simple_addresses(root)
#print(f"Number of simple addresses (NAdd-s): {simple_address_count}")

Number of simple addresses (NAdd-s): 5


## Number of Simple Address (NAdd-s) with the function name and position of & in parameter

In [39]:
def get_function_calls(root):
    function_calls = {}
    # Find all function calls
    calls = root.findall('.//src:call', namespaces=NAMESPACE_SRC)
    # Iterate over each function call
    for call in calls:
        # Extract function name
        function_name = call.find('.//src:name', namespaces=NAMESPACE_SRC).text
        # Extract arguments from the call
        arguments = call.findall('.//src:argument', namespaces=NAMESPACE_SRC)
        # List to store parameter indices with address operator
        address_parameters = []
        # Iterate over each argument
        for index, argument in enumerate(arguments, start=1):
            # Check if the argument has an address operator (&)
            address_operator = argument.find('.//src:operator', namespaces=NAMESPACE_SRC)
            if address_operator is not None and address_operator.text == '&' :
                address_parameters.append(index)
        # Store the address parameters for this function call
        if function_name not in function_calls:
            function_calls[function_name] = address_parameters
        else:
            function_calls[function_name].extend(address_parameters)

    return function_calls

def print_address_parameters(function_calls):
    for function_name, parameters in function_calls.items():
        if parameters:
            parameter_indices = ', '.join(str(param) for param in parameters)
            print(f"{function_name} has addresses for pointers in param {parameter_indices}.")
        else:
            print(f"{function_name} does not have addresses for pointers.")

# Example usage
#function_calls = get_function_calls(root)
#print_address_parameters(function_calls)

func1 has addresses for pointers in param 2, 3.
func2 has addresses for pointers in param 1, 2.
func3 has addresses for pointers in param 1.


## 9. Number of Array Pointers (NPt-ds)

In [99]:
def count_array_pointers(root):
    array_pointer_count = 0
    # Find all function parameters
    parameters = root.findall('.//src:parameter', namespaces=NAMESPACE_SRC)
    # Iterate over each parameter
    for parameter in parameters:
        # Check if the parameter declaration contains an array identifier
        parameter_name = parameter.find('.//src:decl/src:name', namespaces=NAMESPACE_SRC)
        if parameter_name is not None and parameter_name.find('src:index', namespaces=NAMESPACE_SRC) is not None:
            array_pointer_count += 1
    return array_pointer_count

# Example usage
#array_pointer_count = count_array_pointers(root)
#print(f"Number of array pointers (NPt-ds): {array_pointer_count}")

Number of array pointers (NPt-ds): 3


## Number of Array Pointers (NPt-ds) with the function name and position of [] in parameter

In [98]:
def find_array_parameters_with_fun_name(root):
    # Initialize a dictionary to store parameter positions with array identifiers
    array_parameters = {}
    # Find all function declarations
    functions = root.findall('.//src:function', namespaces=NAMESPACE_SRC)
    # Iterate over each function declaration
    for function in functions:
        function_name = function.find('./src:name', namespaces=NAMESPACE_SRC).text
        parameters = function.findall('.//src:parameter', namespaces=NAMESPACE_SRC)
        array_positions = []
        # Iterate over each parameter
        for idx, parameter in enumerate(parameters, start=1):
            parameter_name = parameter.find('.//src:decl/src:name', namespaces=NAMESPACE_SRC)
            if parameter_name is not None and parameter_name.find('src:index', namespaces=NAMESPACE_SRC) is not None:
                array_positions.append(idx)
        if array_positions:
            array_parameters[function_name] = array_positions
    return array_parameters

# Example usage
#array_parameters = find_array_parameters_with_fun_name(root)
#if not array_parameters:
#    print("No array parameters found in any function.")
#else:
#    for function_name, positions in array_parameters.items():
#        positions_str = ', '.join(map(str, positions))
#        print(f"{function_name} has arrays in param {positions_str}")

printArray has arrays in param 1
printArrays has arrays in param 1, 3


## 10. Number of Array Address (NAdd-ds)
### This is complicated and NOT DONE

In [46]:
def extract_function_signatures(xml_file):
    function_signatures = {}
    # Extract function signatures
    for function in root.findall('.//src:function', namespaces=NAMESPACE_SRC):
        function_name = function.find("./src:name", namespaces=NAMESPACE_SRC).text
        parameters = []
        for param in function.findall('.//src:parameter', namespaces=NAMESPACE_SRC):
            param_type_element = param.find("./src:decl/src:type/src:name", namespaces=NAMESPACE_SRC)
            param_type = param_type_element.text
            # Check if the parameter type has an index (indicating an array)
            if param.find("./src:decl//src:index", namespaces=NAMESPACE_SRC) is not None:
                param_type += "[]"
            parameters.append(param_type)
        function_signatures[function_name] = parameters
    return function_signatures

def extract_function_calls(xml_file):
    function_calls = []
    # Extract function calls
    for call in root.findall('.//src:call', namespaces=NAMESPACE_SRC):
        function_name = call.find("./src:name", namespaces=NAMESPACE_SRC).text
        arguments = []
        # Extract arguments from the argument list
        for argument in call.findall('.//src:argument', namespaces=NAMESPACE_SRC):
            # Extract the expression within the argument
            expr = argument.find("./src:expr", namespaces=NAMESPACE_SRC)
            if expr is not None:
                # Get the text of the expression
                expr_text = ''.join(expr.itertext())
                arguments.append(expr_text)
        function_calls.append((function_name, arguments))
    return function_calls

def extract_array_addresses(root):
    function_signatures = extract_function_signatures(root)
    function_calls = extract_function_calls(root)
    print("printing fun_sign:", function_signatures)
    print("printing fun_calls:", function_calls)
    array_addresses_count = 0
    for function_name, parameters in function_signatures.items():
        for param in parameters:
            # Check if the parameter is an array
            if "[]" in param:
                for called_function, arguments in function_calls:
                    # Check if the array parameter name is used as an argument in function calls
                    if function_name == called_function and param in arguments:
                        array_addresses_count += 1
    return array_addresses_count

# Example usage
#array_addresses_count = extract_array_addresses(root)
#print(f"Number of Array Addresses (NAdd-ds): {array_addresses_count}")

printing fun_sign: {'printArray': ['int[]', 'int'], 'printArrays': ['int[]', 'int', 'char[]', 'int'], 'main': []}
printing fun_calls: [('printArray', ['array', '5']), ('printArrays', ['array1', '5', 'array2', '3'])]
Number of Array Addresses (NAdd-ds): 0


## 11. Number of Structs (NStr)

In [48]:
def count_structs(root):
    # Initialize a counter for structs
    struct_count = 0
    # Iterate through lines in the XML file
    for line in root.findall('.//src:struct', namespaces=NAMESPACE_SRC):
        # Increment struct count for each occurrence of 'struct'
        struct_count += 1
    return struct_count

# Example usage
#struct_count = count_structs(root)
#print(f"Number of Structs (NStr): {struct_count}")

Number of Structs (NStr): 2


## 12. Number of Struct Members (NStrM)

In [49]:
def extract_struct_declarations(root):
    struct_declarations = {}
    # Extract struct declarations
    for struct in root.findall('.//src:struct', namespaces=NAMESPACE_SRC):
        struct_name = struct.find("./src:name", namespaces=NAMESPACE_SRC).text
        members = []
        # Extract members within the struct
        for member in struct.findall('.//src:decl', namespaces=NAMESPACE_SRC):
            member_name = member.find("./src:name", namespaces=NAMESPACE_SRC).text
            members.append(member_name)
        struct_declarations[struct_name] = members
    return struct_declarations

def count_struct_members(root):
    struct_declarations = extract_struct_declarations(root)
    print("printing struct_declarations: ",struct_declarations)
    total_members = 0
    # Count the number of struct members for each struct
    for struct_name, members in struct_declarations.items():
        total_members += len(members)
    return total_members

# Example usage
#total_struct_members = count_struct_members(root)
#print(f"Number of Struct Members (NStrM): {total_struct_members}")

printing struct_declarations:  {'Point': ['x', 'y'], 'Student': ['id', 'name', 'gpa']}
Number of Struct Members (NStrM): 5


## 13. Number of Struct Typedefs (NStrT)

In [51]:
def count_struct_typedefs(root):
    typedefs = []
    # Extract typedef commands defining structs
    for typedef in root.findall('.//src:typedef', namespaces=NAMESPACE_SRC):
        # Check if the typedef is for a struct
        if typedef.find('.//src:struct', namespaces=NAMESPACE_SRC) is not None:
            typedef_name = typedef.find('./src:name', namespaces=NAMESPACE_SRC).text
            print("printing ",typedef_name)
            typedefs.append(typedef_name)
    return len(typedefs)

# Example usage
#struct_typedef_count = count_struct_typedefs(root)
#print(f"Number of Struct Typedefs (NStrT): {struct_typedef_count}")

printing  Person
printing  Point
Number of Struct Typedefs (NStrT): 2


## 14.  Number of Struct Instances (NStrI)

In [53]:
def extract_typedef_aliases(root):
    typedef_aliases = []
    # Extract typedef commands defining structs
    for typedef in root.findall('.//src:typedef', namespaces=NAMESPACE_SRC):
        # Check if the typedef is for a struct
        if typedef.find('.//src:struct', namespaces=NAMESPACE_SRC) is not None:
            typedef_alias = typedef.find('./src:name', namespaces=NAMESPACE_SRC).text
            typedef_aliases.append(typedef_alias)
    return typedef_aliases

def count_struct_instances(root):
    typedef_aliases = extract_typedef_aliases(root)
    struct_instance_count = {alias: 0 for alias in typedef_aliases}
    #print("printing struct_instance_count ",struct_instance_count)
    # Count instances of typedef aliases in the code
    for line in root.findall('.//src:block//src:decl/src:type/src:name', namespaces=NAMESPACE_SRC):
        line_text = line.text.strip() if line.text else ''
        for alias in typedef_aliases:
            if line_text.startswith(alias):
                struct_instance_count[alias] += 1
    #return struct_instance_count
    return sum(struct_instance_count.values())

# Example usage
#struct_instance_count = count_struct_instances(root)
#print("Number of Struct Instances (NStrI):")
#for alias, count in struct_instance_count.items():
#    print(f"{alias}: {count}")
#total_count = sum(struct_instance_count.values())
#print("Total count:", total_count)

Number of Struct Instances (NStrI):
Person: 1
Point: 1
Total count: 2


## 15. Number of Struct Calls (NStrC)

In [55]:
def extract_struct_instances(root):
    struct_instances = []
    # Extract instances of the heterogeneous data structure
    for instance in root.findall('.//src:struct', namespaces=NAMESPACE_SRC):
        type_name = instance.find('./src:name', namespaces=NAMESPACE_SRC).text
        struct_instances.append(type_name)
    return struct_instances

def count_struct_calls(root):
    struct_instances = extract_struct_instances(root)
    struct_inst_name={}
    # Search for each instance through the source code and count the occurrences
    declarations =root.findall('.//src:block//src:decl', namespaces=NAMESPACE_SRC)
    for call in declarations:
        declaration_type_element = call.find('./src:type/src:name', namespaces=NAMESPACE_SRC)
        if declaration_type_element is not None:
            declaration_type = declaration_type_element.text
            if declaration_type in struct_instances:
                inst_name_element = call.find('./src:name', namespaces=NAMESPACE_SRC)
                if inst_name_element is not None:
                    inst_name = inst_name_element.text
                    if declaration_type in struct_inst_name:
                        struct_inst_name[declaration_type].append(inst_name)
                    else:
                        struct_inst_name[declaration_type] = [inst_name]
            
    print("printing struct_inst_name: ",struct_inst_name)
    struct_call_count = {instance: 0 for instance in struct_instances}
    for struct_name, inst_names in struct_inst_name.items():
        print("printing inst_names: ", inst_names)
        for inst in inst_names:
            for check_inst in root.findall('.//src:expr_stmt/src:expr//src:name', namespaces=NAMESPACE_SRC):
                if check_inst.text == inst:
                    #print("asDfgbvfds")
                    struct_call_count[struct_name] += 1
    #return struct_call_count
    return sum(struct_call_count.values())

# Example usage
#struct_call_count = count_struct_calls(root)
#print("Number of Struct Calls (NStrC):")
#for instance, count in struct_call_count.items():
#    print(f"{instance}: {count}")
#total_count = sum(struct_call_count.values())
#print("Total count:", total_count)

printing struct_inst_name:  {'Point': ['p1'], 'Student': ['s1', 's2']}
printing inst_names:  ['p1']
printing inst_names:  ['s1', 's2']
Number of Struct Calls (NStrC):
Point: 2
Student: 7
Total count: 9


## 16. Number of Recursive Functions (NFRec)

In [111]:
def extract_function_bodies_to_check_if_recursive(xml_file):
    #srcml --xpath="//src:function//src:block//src:call/src:name" recursionExample.cpp
    function_bodies = {}
    for function in root.findall('.//src:function', namespaces=NAMESPACE_SRC):
        function_name = function.find("./src:name", namespaces=NAMESPACE_SRC).text
        block = function.find('.//src:block', namespaces=NAMESPACE_SRC)
        if block is not None:
            func_calls_in_body = block.findall('.//src:call/src:name', namespaces=NAMESPACE_SRC)
            func_calls = [call.text for call in func_calls_in_body]
            function_bodies[function_name] = func_calls
    return function_bodies

def count_recursive_functions(root):
    function_bodies = extract_function_bodies_to_check_if_recursive(root)
    print("printing function_bodies:",function_bodies)
    recursive_function_count = 0
    for function_name, body in function_bodies.items():
        #print("printing funcName: ",function_name)
        #print("printing body: ",body)
        # Check if the function name appears in its own body
        if function_name in body:
            recursive_function_count += 1
    return recursive_function_count

# Example usage
#recursive_function_count = count_recursive_functions(root)
#print(f"Number of Recursive Functions (NFRec): {recursive_function_count}")

printing function_bodies: {'factorial': ['factorial'], 'fibonacci': ['fibonacci', 'fibonacci'], 'power': ['power'], 'recursiveFuncWith2FuncReturns': ['recursiveFuncWith2FuncReturns', 'recursiveFuncWith2FuncReturns'], 'main': ['factorial', 'fibonacci', 'power', 'recursiveFuncWith2FuncReturns', 'recursiveFuncWith2FuncReturns']}
Number of Recursive Functions (NFRec): 4


## 17. Number of Recursive Functions Calls (NCRec)

In [139]:
def count_recursive_function_calls(root):
    function_bodies = extract_function_bodies_to_check_if_recursive(root)
    #print("printing function_bodies:",function_bodies)
    recursive_functions = [func_name for func_name, body in function_bodies.items() if func_name in body]
    print("printing recursive_functions: ",recursive_functions)
    recursive_function_call_count = 0
    for func_name, body in function_bodies.items():
        if func_name not in recursive_functions:
            for recursive_func_name in recursive_functions:
                #print("printing recursive_func_name: ",recursive_func_name)
                #print("printing body: ",body)
                recursive_function_call_count += body.count(recursive_func_name)

    return recursive_function_call_count

# Example usage
#recursive_function_call_count = count_recursive_function_calls(root)
#print(f"Number of Recursive Function Calls (NCRec): {recursive_function_call_count}")

printing recursive_functions:  ['factorial', 'fibonacci', 'power', 'recursiveFuncWith2FuncReturns']
Number of Recursive Function Calls (NCRec): 5


## 18. Number of Parameters in Conditional (NIFPar)

In [194]:
def extract_function_bodies_as_et_object(root):
    function_bodies = {}
    for function in root.findall('.//src:function', namespaces=NAMESPACE_SRC):
        function_name = function.find("./src:name", namespaces=NAMESPACE_SRC).text
        block = function.find('.//src:block', namespaces=NAMESPACE_SRC)
        if block is not None:
            #function_body = ET.tostring(block, encoding='unicode', method='text').strip()
            #function_bodies[function_name] = function_body
            function_bodies[function_name] = block
    return function_bodies

def extract_function_parameters(root):
    function_parameters = {}
    for function in root.findall('.//src:function', namespaces=NAMESPACE_SRC):
        function_name = function.find("./src:name", namespaces=NAMESPACE_SRC).text
        parameters = function.findall('.//src:parameter/src:decl/src:name', namespaces=NAMESPACE_SRC)
        parameter_names = [param.text for param in parameters]
        function_parameters[function_name] = parameter_names
    return function_parameters

def count_parameters_in_conditional(root):
    function_bodies = extract_function_bodies_as_et_object(root)
    function_parameters = extract_function_parameters(root)
    #print("printing function_bodies: ",ET.tostring(function_bodies, encoding='unicode', method='text').strip())
    #print("printing function_parameters: ",function_parameters)
    recursive_functions = []  # List to store names of recursive functions
    # Identify recursive functions
    for func_name, body in function_bodies.items():
        func_calls_in_body = body.findall('.//src:call/src:name', namespaces=NAMESPACE_SRC)
        func_calls = [call.text for call in func_calls_in_body]
        if func_name in func_calls:
            recursive_functions.append(func_name)
    print("printing recursive_functions: ",recursive_functions)
    conditional_parameter_count = 0
    for function_name, body in function_bodies.items():
        if function_name in recursive_functions:
            parameters = function_parameters.get(function_name, [])
            if parameters:
                for param in parameters:
                    #print("printing param: ",param)
                    check_for_param = root.findall('.//src:function[src:name="' + function_name + '"]//src:if/src:condition//src:name', namespaces=NAMESPACE_SRC)
                    for condition in check_for_param:
                        if param in condition.text:
                            #print("Parameter", param, "found in function", function_name, "within condition:", condition_text)
                            conditional_parameter_count += 1
    return conditional_parameter_count

# Example usage
#conditional_parameter_count = count_parameters_in_conditional(root)
#print(f"Number of Parameters in Conditional (NIFPar): {conditional_parameter_count}")

printing recursive_functions:  ['factorial', 'fibonacci', 'power', 'recursiveFuncWith2FuncReturns']
Number of Parameters in Conditional (NIFPar): 5


## 19. Number of Recursive Returns (NRRec)

In [160]:
def extract_function_bodies(root):
    function_bodies = {}
    for function in root.findall('.//src:function', namespaces=NAMESPACE_SRC):
        function_name = function.find("./src:name", namespaces=NAMESPACE_SRC).text
        block = function.find('.//src:block', namespaces=NAMESPACE_SRC)
        if block is not None:
            function_body = ET.tostring(block, encoding='unicode')
            function_bodies[function_name] = function_body
    return function_bodies

def count_recursive_returns(root):
    function_bodies = extract_function_bodies(root)
    recursive_functions = []  # List to store names of recursive functions
    #print("printing function_bodies: ",function_bodies)
    # Identify recursive functions
    for func_name, body in function_bodies.items():
        func_calls_in_body = ET.fromstring(body).findall('.//src:call/src:name', namespaces=NAMESPACE_SRC)
        func_calls = [call.text for call in func_calls_in_body]
        if func_name in func_calls:
            recursive_functions.append(func_name)
    print("printing recursive_functions: ",recursive_functions)
    recursive_return_count = 0
    for func_name, body in function_bodies.items():
        return_tags = ET.fromstring(body).findall('.//src:return', namespaces=NAMESPACE_SRC)
        if func_name in recursive_functions:
            for return_tag in return_tags:
                call_tag = return_tag.find('.//src:call/src:name', namespaces=NAMESPACE_SRC)
                #print("printing call_tag: ",call_tag)
                if call_tag is not None and call_tag.text == func_name:
                    recursive_return_count += 1
    return recursive_return_count

# Example usage
#recursive_return_count = count_recursive_returns(root)
#print(f"Number of Recursive Returns (NRRec): {recursive_return_count}")

printing recursive_functions:  ['factorial', 'fibonacci', 'power', 'recursiveFuncWith2FuncReturns']
Number of Recursive Returns (NRRec): 5


## 20. Number of Non-Recursive Returns (NRNRec)

In [125]:
def count_non_recursive_returns(root):
    function_bodies = extract_function_bodies(root)
    recursive_functions = []  # List to store names of recursive functions
    #print("printing function_bodies: ",function_bodies)
    # Identify recursive functions
    for func_name, body in function_bodies.items():
        func_calls_in_body = ET.fromstring(body).findall('.//src:call/src:name', namespaces=NAMESPACE_SRC)
        func_calls = [call.text for call in func_calls_in_body]
        if func_name in func_calls:
            recursive_functions.append(func_name)
    print("printing recursive_functions: ",recursive_functions)
    non_recursive_return_count = 0
    for func_name, body in function_bodies.items():
        return_tags = ET.fromstring(body).findall('.//src:return', namespaces=NAMESPACE_SRC)
        if func_name in recursive_functions:
            for return_tag in return_tags:
                call_tag = return_tag.find('.//src:call/src:name', namespaces=NAMESPACE_SRC)
                #print("printing call_tag: ",call_tag)
                if call_tag is None or call_tag.text != func_name:
                    non_recursive_return_count += 1
    return non_recursive_return_count

# Example usage
#non_recursive_return_count = count_non_recursive_returns(root)
#print(f"Number of Non-Recursive Returns (NRNRec): {non_recursive_return_count}")

printing recursive_functions:  ['factorial', 'fibonacci', 'power', 'recursiveFuncWith2FuncReturns']
Number of Non-Recursive Returns (NRNRec): 4


## 21. Number of Malloc (NMalloc)

In [67]:
def count_malloc_functions(root):
    malloc_count = 0
    # Search for malloc functions in the source code
    for malloc_call in root.findall('.//src:call[src:name="malloc"]', namespaces=NAMESPACE_SRC):
        malloc_count += 1
    return malloc_count

# Example usage
#malloc_count = count_malloc_functions(root)
#print(f"Number of Malloc (NMalloc): {malloc_count}")

Number of Malloc (NMalloc): 3


## 22. Number of Sizeof (NSizeof)

In [70]:
def count_sizeof(root):
    sizeof_count = 0
    # Search for sizeof keyword throughout the source code
    for sizeof in root.findall('.//src:sizeof', namespaces=NAMESPACE_SRC):
        sizeof_count += 1
    return sizeof_count

# Example usage
#sizeof_count = count_sizeof(root)
#print(f"Number of Sizeof (NSizeof): {sizeof_count}")

Number of Sizeof (NSizeof): 4


## 23. Number of Free (NFree)

In [72]:
def count_free_functions(root):
    free_count = 0
    # Search for occurrences of the 'free' keyword
    for free_call in root.findall('.//src:call[src:name="free"]', namespaces=NAMESPACE_SRC):
        # Increment the count for each occurrence
        free_count += 1
    return free_count

# Example usage
#num_free_functions = count_free_functions(root)
#print("Number of Free Functions (NFree):", num_free_functions)

Number of Free Functions (NFree): 3
